# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Xinyao Niu
###### Python version: 3.6.8
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import join
import subprocess
import pandas as pd
from collections import defaultdict as dd

In [31]:
def bash_call(c:'[str]') -> 'null':
    res = subprocess.check_output(c)
    for line in res.splitlines():
        print(line.decode('utf-8'))

In [32]:
root = join(os.getcwd(),'2019S1-proj1-data')
bash_call(['ls', root])

README.txt
anneal.csv
breast-cancer.csv
car.csv
cmc.csv
headers.txt
hepatitis.csv
hypothyroid.csv
mushroom.csv
nursery.csv
primary-tumor.csv


In [69]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,3-5,yes,2,left,right_up,no,recurrence-events


In [99]:
class bayes_learner():
    
    def __init__(self, path:str):
        self.path = path
        self.df = None
        self.labels = {}
        self.features = {}
        
    # This function should open a data file in csv, and transform it into a usable format 
    def preprocess(self) -> None:
        data = pd.read_csv(self.path, header=None)
        return data
    
    # count the frequency for each different values in an array
    def count_frequency(array:np.array, accumulate:dict={}) -> dict:
        res = dd(lambda:0,accumulate)
        for i in array:
            res[i] += 1
        return res


In [100]:
have = bayes_learner(join(root, 'breast-cancer.csv'))

In [95]:
# This function should build a supervised NB model
def train(data:'DataFrame') -> None:
    count = []

In [ ]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict():
    return

In [ ]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [ ]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    return

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.